In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from collections import OrderedDict
from pylab import rcParams
plt.style.use('ggplot')
defaults = OrderedDict((('width', 8), ('height', 4)))
rcParams['figure.figsize'] = defaults.values()
rcParams['legend.frameon'] = True
rcParams['legend.framealpha'] = 0.8
rcParams['figure.titlesize'] = 'large'
rcParams['font.size'] = 8
rcParams['legend.fontsize'] = 8
rcParams['xtick.labelsize'] = 8
rcParams['ytick.labelsize'] = 8

In this notebook we will apply the Fourier Transform to two common problems in
time series analysis: outlier detection and extrapolation.

## Outlier detection in periodical data

An outlier is an event or observation that deviates from what is considered the expected patter
of other observations in the dataset. In general, if we assume that a dataset of observation is generated by
some stochastic process, an outlier is a measurement so extreme that appears to have been generated
by a different mechanism.

In practice we, an anomalous observation will often indicate some kind of problem: a fraud in a set of credit cards transactions, malfunctioning of a measuring sensor, spikes in web traffic generated by a botnet.

### Dataset

In this lab we will analyize a dataset of the monthly numbers of [sunspots](https://en.wikipedia.org/wiki/Sunspot) recorded by the World Data Center. Sunspots are physical phenomena defined by wikipedia as follows:

> Sunspots are temporary phenomena on the photosphere of the Sun that appear as dark spots compared to surrounding regions. They are areas of reduced surface temperature caused by concentrations of magnetic field flux that inhibit convection. [...] Due to its link to other kinds of solar activity, sunspot occurrence can be used to help predict space weather, the state of the ionosphere, and hence the conditions of short-wave radio propagation or satellite communications. Solar activity (and the solar cycle) have been implicated in global warming, originally the role of the Maunder Minimum of sunspot occurrence in the Little Ice Age in European winter climate. Sunspots themselves, in terms of the magnitude of their radiant-energy deficit, have a weak effect on terrestrial climate in a direct sense. On longer time scales, such as the solar cycle, other magnetic phenomena (faculae and the chromospheric network) correlate with sunspot occurrence.

We provide you a with a univariate timeserie reporting the number of months since January 1749, and the average number of daily sunspots recorded in that month. The data has been acquired from the [WDC-SILSO, Royal Observatory of Belgium, Brussels](http://www.sidc.be/silso/datafiles).

## Goal

In this exercise we will use Fourier analysis to identify abnormal numbers of sunspot observations.

This dataset if particularly interesting for us, because sunsposts phenomena show periodical patterns. FFT can help us identify this cycles, and reason about potential outliers.

**1. Load and visualize the dataset**

Load and plot the `data/sunspot.csv` dataset into a Pandas `DataFrame`. Label the  axes so that `x` reports  the
months since Jan 1749 and y the average number of sun spots that year. Make sure to create a `DatetimeIndex` with monthly dates ranging from 1749-01 onwards

In [ ]:
<your answer here>

**1b. Do you see a periodic phenomenon?  **

In [ ]:
<your answer here>

**2. Smooth the timeseries using a moving average or median**

Recall from the lecture that moving average/median is a form of convolution. One could apply `np.convolve()` to the signal, or use Pandas `rolling()` convenience method.

In [ ]:
<your answer here>

**3. Detect outliers by comparing the smoothed timeseries with the raw one**

We take the difference between the original data and the smoothed timeseries, and label all points larger then a given `threshold` value as outliers. What is a good value of `threshold`?

In [ ]:
<your answer here>

**4a. Can you think of a way of using FFT to smooth the timeseries and label outliers? What could be a limitation of this approach to outlier detection? **

You don't need to actually implement the method, rather come up with some discussion points
to share with the class.

In [ ]:
<your answer here>

**4b.  what is one limitation of the approaches we considered so far?**
    

In [ ]:
<your answer here>

### Intermezzo: timeseries decomposition

We can follow a similar approach to the one we saw in the timeseries module, and decompose the sunspot data into a seasonal, trend and error (or residual) components.

The general equation is the following:

$x(t)=s(t)+m(t)+e(t)$

where:
 * $t$ is the time
 * $x$ is the data
 * $s$ is the seasonal component
 * $e$ is the random error term
 * $m$ is the trend

**5. Use FFT to determine the periodicity of the timeseries**

From question 1b, we know that sunspots activity is cyclical, reaching a maximum roughly every $x$ years. Use Fourier analysis to confirm that. Use the `resample` method from pandas to resample the original dataset to yearly measurements. Peform the same fourier analysis on the monthly data. Do the results match?

In [ ]:
<your answer here>

**6. Find a trend in the timeseries.**

We can use a rolling mean/average with a window equal to the dominant frequency to estimate the timeseries trend.

In [ ]:
<your answer here>

**7 Exploit the previous steps to calculate the error term by combining the seasonality and trend of the timeseries**

Recall that the error term, for an additive model, is defined as $Error = Data - (Trend + Seasonality)$

In [ ]:
<your answer here>

**8. Detect outliers in the error component, and use their index to label the original data.**

One way to label outliers is to perform a simple [IQR test](http://www.purplemath.com/modules/boxwhisk3.htm) to set the threshold.

In [167]:
q95, q5  = error.quantile([.95, .5])
iqr = q95 - q5

We mark the points that fall out of the $(-1.5 IQR, 1.5IQR)$ range as outliers.

In [ ]:
<your answer here>

## Extrapolation (bonus execise)

We can formulate the process of data extrapolation as reconstructing a signal,
follows these steps:
 1. look for a linear trend in the signal (see numpy's `polyfit` function or `signal.detrend` in `scipy`)
 2. detrend the signal
 3. calculate the FFT of the detrended signal
 4. select only frequency higher than a threshold $h$ to restore the signal
 5. restore and extrapolate the signal as $ \sum_i amplitude_i \cdot \cos(2 \cdot \pi \cdot freq_i + phase_i) $
 6. alternatively, restore and extrapolate the signal using the inverse Fourier transform. Is the result you obtained different than 5? If so, why do you think it is the case?

Implement a `fourier_extrapolation` function following 1-6 and use it to extrapolate monthly sunspot activity. Discuss the results.